In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import config
from dataset import INAOEDataset, DFUData

This notebook is used in order to extract the features from the Dataset. Now It is just tested in the INAOE dataset.

In [ ]:
# Estimated Temperature (ET)
def estimated_temperature(hist: np.ndarray, cluster: np.ndarray):
    argmax = np.argmax(hist)
    if argmax < len(cluster) - 1:
        et_hist = hist[[argmax-1, argmax, argmax+1]]
        et_cluster = cluster[[argmax-1, argmax, argmax+1]]
    else:
        et_hist = hist[[argmax-1, argmax]]
        et_cluster = cluster[[argmax-1, argmax]]
    # et_cluster = cluster[[argmax-1, argmax, argmax+2]]
    return np.sum(et_hist * et_cluster) / np.sum(et_hist)

# Estimated Temperature difference (ETD)
def estimated_temperature_difference(left_et_angiosome, right_et_angiosome):
    return np.abs(left_et_angiosome - right_et_angiosome)

# Hot Pot Estimator
def hot_spot_estimator(hist:np.ndarray, cluster:np.ndarray):
    argmax = np.argmax(hist)
    et = estimated_temperature(hist, cluster)
    return np.abs(cluster[argmax] - et)


# INAOE Dataset

In [ ]:
dm_group = True
save_path = 'data/DM' if dm_group else 'data/CG'

dataset = INAOEDataset(config.INAOE_DATASET_DIR, dm_group=dm_group)

In [ ]:
from scipy.stats import skew, kurtosis

def get_metrics(dataset, data:DFUData):
    subjects = dataset.get_subjects_names()
    mean_data = []
    std_data = []
    max_data = []
    min_data = []
    skew_data = []
    kurtosis_data = []
    expected_temerature_data = []
    hot_spot_data = []
    metrics = []
    nrt_c0, nrt_c1, nrt_c2, nrt_c3, nrt_c4, nrt_c5, nrt_c6, nrt_c7 = [], [], [], [], [], [], [], []

    fulldata = None
    for idx in range(len(dataset)):
        subject_data = dataset.get_subject(idx, data).to_numpy()
        subject_data = subject_data[subject_data>18] # Remove background, temperatures lower than 18 are considered outliers!
        mean_data.append(subject_data.mean())
        std_data.append(subject_data.std())
        max_data.append(subject_data.max())
        min_data.append(subject_data.min())
        skew_data.append(skew(subject_data))
        kurtosis_data.append(kurtosis(subject_data))

        hist, _ = np.histogram(subject_data, bins=config.EXPECTED_VALUE_INTERVAL)
    
        # NRTClass
        nrt_class = hist / hist.sum()
        nrt_c0.append(nrt_class[0])
        nrt_c1.append(nrt_class[1])
        nrt_c2.append(nrt_class[2])
        nrt_c3.append(nrt_class[3])
        nrt_c4.append(nrt_class[4])
        nrt_c5.append(nrt_class[5])
        nrt_c6.append(nrt_class[6])
        nrt_c7.append(nrt_class[7])
    
        # Expected Temperature (ET) and Hot Spot Estimator
        expected_temerature_data.append(estimated_temperature(hist, config.EXPECTED_VALUE_CLUSTER))
        hot_spot_data.append(hot_spot_estimator(hist, config.EXPECTED_VALUE_CLUSTER))

    metrics = {'Subject': subjects,
        '{}_mean'.format(data.name): mean_data, 
        '{}_std'.format(data.name): std_data,
        '{}_min'.format(data.name): min_data,
        '{}_max'.format(data.name): max_data,
        '{}_skew'.format(data.name): skew_data,
        '{}_kurtosis'.format(data.name): kurtosis_data,
        '{}_NRT_C0'.format(data.name): nrt_c0,
        '{}_NRT_C1'.format(data.name): nrt_c1,
        '{}_NRT_C2'.format(data.name): nrt_c2,
        '{}_NRT_C3'.format(data.name): nrt_c3,
        '{}_NRT_C4'.format(data.name): nrt_c4,
        '{}_NRT_C5'.format(data.name): nrt_c5,
        '{}_NRT_C6'.format(data.name): nrt_c6,
        '{}_NRT_C7'.format(data.name): nrt_c7,
        '{}_HSE'.format(data.name): hot_spot_data,
        '{}_ET'.format(data.name): expected_temerature_data,
    }

    return pd.DataFrame(metrics)

## MPA

In [ ]:
l_mpa_metrics = get_metrics(dataset, DFUData.L_MPA)
r_mpa_metrics = get_metrics(dataset, DFUData.R_MPA)

mpa_metrics = pd.merge(l_mpa_metrics, r_mpa_metrics, on='Subject')
ETD = estimated_temperature_difference(mpa_metrics['L_MPA_ET'], mpa_metrics['R_MPA_ET'])
mpa_metrics['MPA_ETD'] = ETD

mpa_metrics.to_csv(os.path.join(save_path, 'MPA/MPA_metrics.csv'), index=False)

## LCA

In [ ]:
l_lca_metrics = get_metrics(dataset, DFUData.L_LCA)
r_lca_metrics = get_metrics(dataset, DFUData.R_LCA)

lca_metrics = pd.merge(l_lca_metrics, r_lca_metrics, on='Subject')
ETD = estimated_temperature_difference(lca_metrics['L_LCA_ET'], lca_metrics['R_LCA_ET'])
lca_metrics['LCA_ETD'] = ETD

lca_metrics.to_csv(os.path.join(save_path, 'LCA/LCA_metrics.csv'), index=False)

## LPA

In [ ]:
l_lpa_metrics = get_metrics(dataset, DFUData.L_LPA)
r_lpa_metrics = get_metrics(dataset, DFUData.R_LPA)

lpa_metrics = pd.merge(l_lpa_metrics, r_lpa_metrics, on='Subject')
ETD = estimated_temperature_difference(lpa_metrics['L_LPA_ET'], lpa_metrics['R_LPA_ET'])
lpa_metrics['LPA_ETD'] = ETD

lpa_metrics.to_csv(os.path.join(save_path, 'LPA/LPA_metrics.csv'), index=False)

## MCA

In [ ]:
l_mca_metrics = get_metrics(dataset, DFUData.L_MCA)
r_mca_metrics = get_metrics(dataset, DFUData.R_MCA)

mca_metrics = pd.merge(l_mca_metrics, r_mca_metrics, on='Subject')
ETD = estimated_temperature_difference(mca_metrics['L_MCA_ET'], mca_metrics['R_MCA_ET'])
mca_metrics['MCA_ETD'] = ETD

mca_metrics.to_csv(os.path.join(save_path, 'MCA/MCA_metrics.csv'), index=False)

## Full Foot

In [ ]:
l_metrics = get_metrics(dataset, DFUData.L)
r_metrics = get_metrics(dataset, DFUData.R)

full_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(full_metrics['L_ET'], full_metrics['R_ET'])
full_metrics['Foot_ETD'] = ETD

full_metrics.to_csv(os.path.join(save_path, 'Full/Full_metrics.csv'), index=False)

In [ ]:
full_metrics

## Merge

In [ ]:
from functools import reduce 

feature_dataset_df = reduce(lambda left,right: pd.merge(left,right,on='Subject'), [mpa_metrics, lca_metrics, lpa_metrics, mca_metrics, full_metrics])

group = 'dm' if dm_group else 'cg'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format(group)), index=False)

## TCI

In [ ]:
def temperature_cg_values(angiosomes: DFUData):
    '''
        Returns the control temperature for the given angiosomes based on the paper
    '''
    data = [-1, -1, 26.1, 26.1, 25.7, 25.7, 26.4, 26.4, 25.8, 25.8]
    if isinstance(angiosomes, list):
        return np.array(list(map(lambda x: data[x.value], angiosomes)))
    else:
        return data[angiosomes.value]

def thermal_change_index(features_df: pd.DataFrame):
    feet_angiosomes = [ [DFUData.L_LCA, DFUData.L_LPA, DFUData.L_MCA, DFUData.L_MPA],
                        [DFUData.R_LCA, DFUData.R_LPA, DFUData.R_MCA, DFUData.R_MPA] ]

    tci = []
    for foot_angiosomes in feet_angiosomes:
        columns = list(map(lambda x: '{}_mean'.format(x.name), foot_angiosomes))
        control_temperature = temperature_cg_values(foot_angiosomes)
        tci.append((np.abs(control_temperature - features_df[columns].to_numpy())).mean(axis=1))
    
    return tci

left_tci, right_tci = thermal_change_index(feature_dataset_df)

feature_dataset_df['L_TCI'] = left_tci
feature_dataset_df['R_TCI'] = right_tci    

group = 'dm' if dm_group else 'cg'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format(group)), index=False)

In [ ]:
feature_dataset_df

## Dataset

Merge the Control Group and the Experimental Group (Non-Diabetic and Diabetic).

In [ ]:
save_dir = 'data'
cg_df = pd.read_csv(os.path.join(save_dir, 'CG/cg_metrics.csv'))
dm_df = pd.read_csv(os.path.join(save_dir, 'DM/dm_metrics.csv'))

pd.concat([cg_df,dm_df], axis=0).to_csv(os.path.join(save_dir, 'dfu_features_dataset.csv'), index=False)

# IACTEC